In [3]:
import skopt
# !pip install scikit-optimize if  necessary
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer  

import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow
from tensorflow.python.keras import backend as K

In [5]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [6]:
path = 'https://raw.githubusercontent.com/FabioHodo/EIB2_ClimateRisk/main/cleaned%20data/data_imputed/imputed_knn/'
df = pd.read_csv(path +'Emissions_GHG_(fromCAIT).csv',index_col='Unnamed: 0' )
df

,Country,Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Albania,ALB,11.40,9.04,6.99,6.93,7.77,7.53,7.25,6.54,...,8.54,8.60,8.88,9.33,8.94,9.05,9.54,9.48,9.33,9.90
1,Algeria,DZA,96.00,94.87,94.50,102.03,99.78,103.99,105.30,109.39,...,156.40,166.21,167.32,179.68,189.89,193.59,203.79,211.66,209.41,211.50
2,Angola,AGO,150.61,151.28,151.61,152.39,152.54,156.90,161.39,156.01,...,173.99,180.66,186.83,220.92,185.95,203.34,185.10,207.66,182.19,176.85
3,Antigua and Barbuda,ATG,0.34,0.35,0.43,0.41,0.42,0.44,0.47,0.50,...,1.06,2.02,1.13,1.07,1.29,1.06,1.08,1.10,1.14,1.17
4,Argentina,ARG,323.84,328.83,333.22,335.73,340.81,341.39,344.46,346.24,...,468.33,452.69,440.34,450.10,464.88,462.89,456.74,460.21,469.73,469.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,VUT,0.47,0.48,0.50,0.53,0.54,0.54,0.57,0.57,...,0.70,0.70,0.71,0.75,0.75,0.75,0.81,0.81,0.84,0.84
163,Venezuela,VEN,303.14,304.92,308.93,315.26,318.02,326.54,331.00,337.59,...,265.49,261.47,291.82,323.30,343.49,373.61,359.46,352.14,352.08,338.14
164,Vietnam,VNM,25.63,28.49,31.00,37.69,43.31,49.46,56.03,63.80,...,209.45,231.71,251.16,235.76,237.24,249.25,263.12,306.26,320.67,319.54
165,Zambia,ZMB,485.43,484.96,484.28,483.00,481.44,478.95,444.46,405.54,...,471.05,412.65,460.49,432.46,455.21,468.32,440.83,473.30,495.17,495.99


In [7]:
def makeMonthly(country, scale = 20):
    '''
    Turn the yearly data into monthly using Brownian bridge algorithm
    Parameter:
        country: a string for the country name
        scale: an integer for the data scaling
    Return:
        the scaled monthly data
    '''
    #AnnualVal = Albania.loc["1990":"2017"]
    AnnualVal = country*scale
    n = (len(AnnualVal)-1)*12
    monthly = np.zeros(n+1)
    monthlyVal = np.zeros(n)
    for i in range(1, len(AnnualVal)+1):
        monthly[(i-1)*12] = AnnualVal[i-1]

    for i in range(1, len(AnnualVal)): #1-4
        for j in range ((i-1)*12+1, (i)*12): #0 to  47
            monthly[j] = monthly[j-1] + (monthly[(i)*12] - monthly[j-1])*(1/((i*12)-(j-1)))
            + (((i*12-j)*(1)/(i*12-(j-1)))**(1/2))*np.random.randn()
            monthlyVal[j-1] = monthly[j] -monthly[j-1]
    return monthly/scale

In [8]:
country='Argentina'
data = df[df['Country']==country]
data = data.loc[:,"1990":"2017"].T
monthly = makeMonthly(np.array(data), 10)
monthly = np.reshape(monthly, (-1, 1))

In [9]:
# Feature Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(monthly)
#split train and test dataset
train_size = int(len(data)*0.7)
test_size = len(data)-train_size
train_data, test_data = data[0:train_size,:],data[train_size:len(data),:]

In [10]:
def create_dataset(dataset,timestep = 10):
    data_x,data_y = [],[]
    for i in range(len(dataset)-timestep-1):
        a = dataset[i:i+timestep]
        data_x.append(a)
        data_y.append(dataset[i+timestep])
    return np.array(data_x),np.array(data_y)

In [11]:
x_train, y_train = create_dataset(train_data,timestep = 10)
x_test, y_test = create_dataset(test_data,timestep = 10)

In [12]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [24]:
dim_learning_rate = Real(low=1e-4, high=1e-2, prior='log-uniform',
                         name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_input_nodes = Integer(low=1, high=512, name='num_input_nodes')
dim_num_dense_nodes = Integer(low=1, high=28, name='num_dense_nodes')
dim_activation = Categorical(categories=['relu', 'sigmoid'],
                             name='activation')
dim_batch_size = Integer(low=1, high=128, name='batch_size')
dim_adam_decay = Real(low=1e-6,high=1e-2,name="adam_decay")

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_input_nodes,
              dim_num_dense_nodes,
              dim_activation,
              dim_batch_size,
              dim_adam_decay
             ]
default_parameters = [1e-3, 1 ,512, 13, 'relu',64, 1e-3]

In [38]:
from tensorflow.keras.optimizers import Adam
def create_model(learning_rate, num_dense_layers,num_input_nodes,
                 num_dense_nodes, activation, adam_decay):
    # Create the Stacked LSTM model
    model = Sequential()

    model.add(LSTM(num_input_nodes, return_sequences = True, input_shape = (x_train.shape[1], 1),
                  activation=activation)
             )
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))
    
    adam = Adam(lr=learning_rate, decay= adam_decay)
    model.compile(optimizer = adam, loss = 'mean_squared_error',metrics=['accuracy'])
    
    return model

In [39]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_input_nodes, 
            num_dense_nodes,activation, batch_size,adam_decay):

    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_input_nodes=num_input_nodes,
                         num_dense_nodes=num_dense_nodes,
                         activation=activation,
                         adam_decay=adam_decay
                        )
    

    #named blackbox becuase it represents the structure
    blackbox = model.fit(x=x_train,
                        y=y_train,
                        epochs=50,
                        batch_size=batch_size,
                        validation_split=0.15,
                        )
    #return the validation accuracy for the last epoch.
    val_loss = blackbox.history['val_loss'][-1]

    
    # the optimizer aims for the lowest score, so we return our negative accuracy
    return val_loss

In [40]:
fitness(x=default_parameters)

Epoch 1/50
3/3 [==============================] - 3s 419ms/step - loss: 0.1351 - accuracy: 0.0000e+00 - val_loss: 0.8225 - val_accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 162ms/step - loss: 0.1193 - accuracy: 0.0000e+00 - val_loss: 0.7500 - val_accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 156ms/step - loss: 0.1006 - accuracy: 0.0000e+00 - val_loss: 0.6490 - val_accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 153ms/step - loss: 0.0777 - accuracy: 0.0000e+00 - val_loss: 0.4983 - val_accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 150ms/step - loss: 0.0527 - accuracy: 0.0000e+00 - val_loss: 0.2919 - val_accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 150ms/step - loss: 0.0439 - accuracy: 0.0000e+00 - val_loss: 0.1776 - val_accuracy: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 0s 151ms/step - loss: 0.0479 - accuracy: 0.0000e+00 - val_loss

0.1686353087425232

In [ ]:
%%time
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=40,
                            x0=default_parameters)

Epoch 1/50
3/3 [==============================] - 3s 436ms/step - loss: 0.1354 - accuracy: 0.0000e+00 - val_loss: 0.8276 - val_accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 153ms/step - loss: 0.1205 - accuracy: 0.0000e+00 - val_loss: 0.7549 - val_accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 155ms/step - loss: 0.1002 - accuracy: 0.0000e+00 - val_loss: 0.6507 - val_accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 150ms/step - loss: 0.0754 - accuracy: 0.0000e+00 - val_loss: 0.5077 - val_accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 156ms/step - loss: 0.0548 - accuracy: 0.0000e+00 - val_loss: 0.3160 - val_accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 145ms/step - loss: 0.0397 - accuracy: 0.0000e+00 - val_loss: 0.1536 - val_accuracy: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 0s 140ms/step - loss: 0.0479 - accuracy: 0.0000e+00 - val_loss

Epoch 4/50
3/3 [==============================] - 0s 160ms/step - loss: 0.0435 - accuracy: 0.0000e+00 - val_loss: 0.2216 - val_accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 163ms/step - loss: 0.0489 - accuracy: 0.0000e+00 - val_loss: 0.2666 - val_accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 148ms/step - loss: 0.0542 - accuracy: 0.0000e+00 - val_loss: 0.4072 - val_accuracy: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 0s 161ms/step - loss: 0.0492 - accuracy: 0.0000e+00 - val_loss: 0.1813 - val_accuracy: 0.0000e+00
Epoch 8/50
3/3 [==============================] - 0s 158ms/step - loss: 0.0523 - accuracy: 0.0000e+00 - val_loss: 0.3532 - val_accuracy: 0.0000e+00
Epoch 9/50
3/3 [==============================] - 0s 162ms/step - loss: 0.0680 - accuracy: 0.0000e+00 - val_loss: 0.4303 - val_accuracy: 0.0000e+00
Epoch 10/50
3/3 [==============================] - 0s 150ms/step - loss: 0.0669 - accuracy: 0.0000e+00 - val_los

Epoch 8/50
14/14 [==============================] - 2s 117ms/step - loss: 0.0607 - accuracy: 0.0000e+00 - val_loss: 0.4313 - val_accuracy: 0.0000e+00
Epoch 9/50
14/14 [==============================] - 2s 114ms/step - loss: 0.0570 - accuracy: 0.0000e+00 - val_loss: 0.3567 - val_accuracy: 0.0000e+00
Epoch 10/50
14/14 [==============================] - 2s 116ms/step - loss: 0.0553 - accuracy: 0.0000e+00 - val_loss: 0.3902 - val_accuracy: 0.0000e+00
Epoch 11/50
14/14 [==============================] - 2s 117ms/step - loss: 0.0575 - accuracy: 0.0000e+00 - val_loss: 0.4156 - val_accuracy: 0.0000e+00
Epoch 12/50
14/14 [==============================] - 2s 115ms/step - loss: 0.0570 - accuracy: 0.0000e+00 - val_loss: 0.4043 - val_accuracy: 0.0000e+00
Epoch 13/50
14/14 [==============================] - 2s 114ms/step - loss: 0.0532 - accuracy: 0.0000e+00 - val_loss: 0.3933 - val_accuracy: 0.0000e+00
Epoch 14/50
14/14 [==============================] - 2s 115ms/step - loss: 0.0591 - accuracy: 0.

3/3 [==============================] - 0s 162ms/step - loss: 0.0720 - accuracy: 0.0000e+00 - val_loss: 0.6221 - val_accuracy: 0.0000e+00
Epoch 12/50
3/3 [==============================] - 0s 147ms/step - loss: 0.0987 - accuracy: 0.0000e+00 - val_loss: 0.4930 - val_accuracy: 0.0000e+00
Epoch 13/50
3/3 [==============================] - 0s 143ms/step - loss: 0.0843 - accuracy: 0.0000e+00 - val_loss: 0.3649 - val_accuracy: 0.0000e+00
Epoch 14/50
3/3 [==============================] - 0s 151ms/step - loss: 0.0851 - accuracy: 0.0000e+00 - val_loss: 0.3806 - val_accuracy: 0.0000e+00
Epoch 15/50
3/3 [==============================] - 0s 149ms/step - loss: 0.0737 - accuracy: 0.0000e+00 - val_loss: 0.4132 - val_accuracy: 0.0000e+00
Epoch 16/50
3/3 [==============================] - 0s 146ms/step - loss: 0.0830 - accuracy: 0.0000e+00 - val_loss: 0.5283 - val_accuracy: 0.0000e+00
Epoch 17/50
3/3 [==============================] - 0s 146ms/step - loss: 0.1147 - accuracy: 0.0000e+00 - val_loss: 0.6

5/5 [==============================] - 1s 138ms/step - loss: 0.0599 - accuracy: 0.0000e+00 - val_loss: 0.4070 - val_accuracy: 0.0000e+00
Epoch 16/50
5/5 [==============================] - 1s 141ms/step - loss: 0.0590 - accuracy: 0.0000e+00 - val_loss: 0.4029 - val_accuracy: 0.0000e+00
Epoch 17/50
5/5 [==============================] - 1s 139ms/step - loss: 0.0552 - accuracy: 0.0000e+00 - val_loss: 0.4029 - val_accuracy: 0.0000e+00
Epoch 18/50
5/5 [==============================] - 1s 138ms/step - loss: 0.0565 - accuracy: 0.0000e+00 - val_loss: 0.4027 - val_accuracy: 0.0000e+00
Epoch 19/50
5/5 [==============================] - 1s 149ms/step - loss: 0.0565 - accuracy: 0.0000e+00 - val_loss: 0.3899 - val_accuracy: 0.0000e+00
Epoch 20/50
5/5 [==============================] - 1s 139ms/step - loss: 0.0587 - accuracy: 0.0000e+00 - val_loss: 0.3969 - val_accuracy: 0.0000e+00
Epoch 21/50
5/5 [==============================] - 1s 139ms/step - loss: 0.0671 - accuracy: 0.0000e+00 - val_loss: 0.4

...

Epoch 1/50
6/6 [==============================] - 5s 482ms/step - loss: 0.1238 - accuracy: 0.0000e+00 - val_loss: 0.6539 - val_accuracy: 0.0000e+00
Epoch 2/50
6/6 [==============================] - 1s 201ms/step - loss: 0.0641 - accuracy: 0.0000e+00 - val_loss: 0.1147 - val_accuracy: 0.0000e+00
Epoch 3/50
6/6 [==============================] - 1s 181ms/step - loss: 0.0453 - accuracy: 0.0000e+00 - val_loss: 0.2782 - val_accuracy: 0.0000e+00
Epoch 4/50
6/6 [==============================] - 1s 158ms/step - loss: 0.0417 - accuracy: 0.0000e+00 - val_loss: 0.2928 - val_accuracy: 0.0000e+00
Epoch 5/50
6/6 [==============================] - 1s 166ms/step - loss: 0.0348 - accuracy: 0.0000e+00 - val_loss: 0.1320 - val_accuracy: 0.0000e+00
Epoch 6/50
6/6 [==============================] - 1s 155ms/step - loss: 0.0334 - accuracy: 0.0000e+00 - val_loss: 0.1505 - val_accuracy: 0.0000e+00
Epoch 7/50
6/6 [==============================] - 1s 152ms/step - loss: 0.0282 - accuracy: 0.0000e+00 - val_loss

/Users/Bruce/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
4/4 [==============================] - 4s 538ms/step - loss: 0.0964 - accuracy: 0.0000e+00 - val_loss: 0.1582 - val_accuracy: 0.0000e+00
Epoch 2/50
4/4 [==============================] - 1s 221ms/step - loss: 0.0477 - accuracy: 0.0000e+00 - val_loss: 0.2795 - val_accuracy: 0.0000e+00
Epoch 3/50
4/4 [==============================] - 1s 184ms/step - loss: 0.0418 - accuracy: 0.0000e+00 - val_loss: 0.2874 - val_accuracy: 0.0000e+00
Epoch 4/50
4/4 [==============================] - 1s 170ms/step - loss: 0.0550 - accuracy: 0.0000e+00 - val_loss: 0.1437 - val_accuracy: 0.0000e+00
Epoch 5/50
4/4 [==============================] - 1s 159ms/step - loss: 0.0896 - accuracy: 0.0000e+00 - val_loss: 0.7596 - val_accuracy: 0.0000e+00
Epoch 6/50
4/4 [==============================] - 1s 157ms/step - loss: 0.1128 - accuracy: 0.0000e+00 - val_loss: 0.3874 - val_accuracy: 0.0000e+00
Epoch 7/50
4/4 [==============================] - 1s 147ms/step - loss: 0.0649 - accuracy: 0.0000e+00 - val_loss

/Users/Bruce/miniforge3/lib/python3.9/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/50
4/4 [==============================] - 4s 507ms/step - loss: 0.1351 - accuracy: 0.0000e+00 - val_loss: 0.8160 - val_accuracy: 0.0000e+00
Epoch 2/50
4/4 [==============================] - 1s 183ms/step - loss: 0.1205 - accuracy: 0.0000e+00 - val_loss: 0.7515 - val_accuracy: 0.0000e+00
Epoch 3/50
4/4 [==============================] - 1s 193ms/step - loss: 0.1037 - accuracy: 0.0000e+00 - val_loss: 0.6651 - val_accuracy: 0.0000e+00
Epoch 4/50
4/4 [==============================] - 1s 173ms/step - loss: 0.0834 - accuracy: 0.0000e+00 - val_loss: 0.5471 - val_accuracy: 0.0000e+00
Epoch 5/50
4/4 [==============================] - 1s 150ms/step - loss: 0.0628 - accuracy: 0.0000e+00 - val_loss: 0.3891 - val_accuracy: 0.0000e+00
Epoch 6/50
4/4 [==============================] - 1s 142ms/step - loss: 0.0473 - accuracy: 0.0000e+00 - val_loss: 0.2334 - val_accuracy: 0.0000e+00
Epoch 7/50
4/4 [==============================] - 1s 153ms/step - loss: 0.0466 - accuracy: 0.0000e+00 - val_loss